## Import libraries 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler
from statistics import mean 

import os
print(os.listdir("../input"))


Using TensorFlow backend.


['train.csv', 'sample_submission.csv', 'test.csv']


## Read Data 

In [2]:
train= pd.read_csv("../input/train.csv")
test = pd.read_csv('../input/test.csv')

In [3]:
y = train['target']
cols = ["target","ID_code"]
x = train.drop(cols,axis=1)

## Create multiple models 

In [4]:
# number of random forest models
num = 200
preds = np.empty([10049, num]) # 10049 is the number of rows in splitted test data
submission = np.empty([test.shape[0], num])
accuracy = []
precision = []
recall = []
auc = []

for i in range(num):
    rus = RandomUnderSampler()
    x_res, y_res = rus.fit_resample(x, y)
    df = pd.DataFrame(pd.np.column_stack([x_res,y_res]))
    df = shuffle(df)
    x_new = df.iloc[:,:200]
    y_new = df.iloc[:,200:]
    
    x_train, x_test, y_train, y_test = train_test_split(x_new,y_new.values.ravel(), test_size=0.25, random_state=0)
    
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.fit_transform(x_test)

    classifier = RandomForestClassifier(n_estimators=100,criterion='entropy')
    classifier.fit(x_train,y_train)
    
    y_pred = classifier.predict(x_test)
    preds[:,i] = y_pred
    print('Running loop no. %s' % (i+1))
    auc.append(metrics.roc_auc_score(y_test, y_pred))
    accuracy.append(metrics.accuracy_score(y_test, y_pred))
    precision.append(metrics.precision_score(y_test, y_pred))
    recall.append(metrics.recall_score(y_test, y_pred))
    
    # for submission
    y_sub = classifier.predict(test.drop("ID_code",axis=1).values)
    submission[:,i] = y_sub

print("AUC: ",mean(auc))
print("Mean Accuracy:",mean(accuracy))
print("Precision:",mean(precision))
print("Recall:",mean(recall))

Running loop no. 1
Running loop no. 2
Running loop no. 3
Running loop no. 4
Running loop no. 5
Running loop no. 6
Running loop no. 7
Running loop no. 8
Running loop no. 9
Running loop no. 10
Running loop no. 11
Running loop no. 12
Running loop no. 13
Running loop no. 14
Running loop no. 15
Running loop no. 16
Running loop no. 17
Running loop no. 18
Running loop no. 19
Running loop no. 20
Running loop no. 21
Running loop no. 22
Running loop no. 23
Running loop no. 24
Running loop no. 25
Running loop no. 26
Running loop no. 27
Running loop no. 28
Running loop no. 29
Running loop no. 30
Running loop no. 31
Running loop no. 32
Running loop no. 33
Running loop no. 34
Running loop no. 35
Running loop no. 36
Running loop no. 37
Running loop no. 38
Running loop no. 39
Running loop no. 40
Running loop no. 41
Running loop no. 42
Running loop no. 43
Running loop no. 44
Running loop no. 45
Running loop no. 46
Running loop no. 47
Running loop no. 48
Running loop no. 49
Running loop no. 50
Running l

In [5]:
submission_rf = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": np.round(submission.mean(axis=1))})

In [6]:
submission_rf.to_csv('submission_rf.csv', index = False)